In [1]:
import numpy as np
import gmsh

'''Inicio la construccion de mi figura y mi mallado'''

gmsh.initialize() #inicia Gmsh
gmsh.model.add('PROBLEMA2') #genera el modelo

#Dimensiones de la figura
L1 = 10
L2 = 5
r = 1
Lc = 0.3     #Esto definira el tamaño de mi mallado en toda la figura menos cerca del cuarto de circulo
Lcr = 0.01  #Refina aun mas el mallado cerca del circulo

#Genero los puntos que definen la chapa
p1 = gmsh.model.geo.addPoint(0, 0, 0, Lc)
p2 = gmsh.model.geo.addPoint(0, L2, 0, Lc) 
p3 = gmsh.model.geo.addPoint(L1, L2, 0, Lc) 
p4 = gmsh.model.geo.addPoint(L1, 0, 0, Lc)

#Genero los puntos que definen el cuarto de circulo
p5 = gmsh.model.geo.addPoint(0,r,0,Lcr)
p6 = gmsh.model.geo.addPoint(r,0,0,Lcr)

#Genero las lineas que definen la chapa
l1 = gmsh.model.geo.addLine(p5, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p6)

#Genero el cuarto de circulo
semi1 = gmsh.model.geo.addCircleArc(p5,p1 ,p6) 

#Genero el contorno uniendo todas las lineas y el cuarto de criculo
C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
C2 = gmsh.model.geo.addCurveLoop([semi1])

S1 = gmsh.model.geo.addPlaneSurface([C1,C2]) #asi se genera la superifice descontando el circulo


#Genero de las condiciones de contorno

TrashNodo = gmsh.model.addPhysicalGroup(0, [p1])       #Esto me permitira mas adelante anclar el centro del circulo para evitar problemas de matriz singular
gmsh.model.setPhysicalName(0, TrashNodo, 'Trash')

Empotradox = gmsh.model.addPhysicalGroup(1, [l1])      #Esto me permitira empotrar en el eje x los nodos de la pared vertical izquierda  
gmsh.model.setPhysicalName(1,Empotradox,'Empotradox')


Empotradoy = gmsh.model.addPhysicalGroup(1, [l4])      #Esto me permitira empotrar en el eje y los nodos de la pared horizontal inferior  
gmsh.model.setPhysicalName(1,Empotradoy,'Empotradoy')


Traccionado = gmsh.model.addPhysicalGroup(1, [l3])      #Esto me permitira definir las fuerzas de traccion en el eje y   
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado') 


Superficie = gmsh.model.addPhysicalGroup(2,[S1])         #Esto genera un grupo fisico con todos los nodos de la superficie
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')


gmsh.model.geo.synchronize() #sincroniza lo que definimos con el modelo que creamos arriba
gmsh.model.mesh.generate(2)


In [2]:
'''En estas lineas recopilo toda la informacion de los nodos que me permite armar las matrices de conectividad (MC) y de posicion de los nodos (MN)'''

NodeInfo = gmsh.model.mesh.get_nodes()
NodeInfo[1].shape
NumeroNodos = NodeInfo[0].shape[0]
MN = NodeInfo[1].reshape(NumeroNodos , 3)


ElementInfo = gmsh.model.mesh.get_elements()
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)
MC = ELEMENTS.reshape([ETAGS.shape[0],3])



In [3]:
'''Calculo las fuerzas sobre cada uno de los elementos de la seccion traccionada
   Una ventaja del metodo implementado para hacerlo es que se independiza de los elementos sean simetricos unos con otros'''

entityTraccionada = gmsh.model.getEntitiesForPhysicalGroup(1, Traccionado) 
Tgroup, Ttraccionada, Ltraccionada = gmsh.model.mesh.getElements(1, entityTraccionada[0])
Ltraccionada = Ltraccionada[0].reshape(Ttraccionada[0].shape[0],2) 
espesor = 1
tension = 1000 #Pa
MNDFuerzas = np.zeros((2*NumeroNodos,1))  #Creo un vector donde ubicare las fuerzas

for l, linea in enumerate(Ltraccionada):  #Con esto calculo las fuerzas sobre cada elemento tal y como explique en el PDF
    n1 = int(linea[0] - 1)
    n2 = int(linea[1] - 1)
    longitud = np.abs(MN[n1,1]-MN[n2,1])  #Calculo las longitudes de cada segmento
    Flocal = tension*longitud*espesor*1/2
    MNDFuerzas[int(2*n1)] += Flocal
    MNDFuerzas[int(2*n2)] += Flocal
    

In [4]:
'''Creo los vectores R y S que permiten aplicar las condiciones de contorno del problema'''


Sx = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradox)[0] - 1)
Sy = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradoy)[0] - 1) +1
Strash1 = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(0,TrashNodo)[0] - 1)
Strash2 = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(0,TrashNodo)[0] - 1) + 1
S = np.hstack([Sx,Sy,Strash1,Strash2])  #Nodos donde aplico condiciones de empotramiento incluyendo al nodo que ya comente antes (el del centro del circulo)
R = np.delete(np.arange(2*NumeroNodos),S) #Nodos donde aplico condiciones de fuerza, sea nula o las ya definidas mas arriba

In [5]:
'''Genero todas las matrices y elementos que luego utilizare'''
E = 30E6
v = 0.3

D = (E/(1-v**2))*np.array([[1,v,0],[v,1,0],[0,0,(0.5*(1-v))]])
Matriz_coordenadas = np.ones((3,3))
Matriz_global = np.zeros([2*len(MN),2*len(MN)])
Matrices_Bloc = []
Areas = []

In [6]:
for l in range (len(MC)):
    
    I,M,J = MC[l]  #Nombro cada uno de los nodos leyendo la matriz MC
    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1) #GMSH enumera desde 1 pero python desde 0 por lo que este paso es necesario
    
    x = np.array([MN[i,0],MN[m,0],MN[j,0]])  #posiciones en x de los nodos
    y = np.array([MN[i,1],MN[m,1],MN[j,1]])  #posiciones en y de los nodos

    Bi = y[1] - y[2] ;   Bm = y[2] - y[0] ;    Bj = y[0] - y[1]      #calculo los coeficientes B y G
    Gi = x[2] - x[1] ;   Gm = x[0] - x[2] ;    Gj = x[1] - x[0]      
    Matriz_coordenadas[:,0] = x
    Matriz_coordenadas[:,1] = y
    
    Area = np.linalg.det(Matriz_coordenadas)/2   #calculo las areas con el determinante de la matriz de coordenadas 
    
    #Calulo las matrices locales y las ubico en la matriz global
    
    Matriz1 = (1/(2*Area)) * np.array([[Bi,0,Bm,0,Bj,0],[0,Gi,0,Gm,0,Gj],[Gi,Bi,Gm,Bm,Gj,Bj]])  
    Matrices_Bloc.append(Matriz1)  
    Matriz_local = espesor * abs(Area) * (np.dot(np.transpose(Matriz1), np.dot(D,Matriz1)))

    A_G = np.array([i*2,i*2+1,m*2,m*2+1,j*2,j*2+1])  #indices
    Matriz_global[np.ix_(A_G,A_G)] += Matriz_local
    
    

In [7]:
Desplazamientos_parcial = np.linalg.solve(Matriz_global[np.ix_(R,R)],MNDFuerzas[R]) #encuentro los desplazamientos que faltan

Desplazamientos_f = np.zeros(len(MN)*2)

o = 0 

#Ubico todos los desplazamientos en sus respectivos nodos 
for i in range(NumeroNodos*2):
    if i in R:
        Desplazamientos_f[i] = Desplazamientos_parcial[o]
        o += 1
        
    if i in S:
        Desplazamientos_f[i] = 0

    
    
Fuerzas_total = np.dot(Matriz_global,Desplazamientos_f)  #Encuentro todas las fuerzas que faltan

In [8]:
#Caluclo de los esfuerzos

Tensiones = []

for p in range(len(MC)):
    
    I,M,J = MC[p]    
    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)

    
    nodos = [2*i,2*i+1,2*m,2*m+1,2*j,2*j+1]
    
    
    A = np.dot(Matrices_Bloc[p],Desplazamientos_f[nodos])
    
    B = np.dot(D,A)

   
    Tensiones.append(B)

Tensiones_ = np.array(Tensiones)



In [9]:
'''agrega todos los datos en el modelo construido al incio'''

Problema2 = gmsh.view.add("Problema2")
problema2 = gmsh.view.addModelData(Problema2, 0, 'PROBLEMA2', 'ElementData', ETAGS, Tensiones_[:,0].reshape(-1,1) , numComponents=1)

In [10]:
'''agrega todos los datos en el modelo construido al incio'''

#Reconstruyo los desplazamientos en un matriz de 3x3 para que el gmsh lo pueda graficar
Desplazamientos = np.zeros((len(MN),3))
for i in range(len(MN)):
    Desplazamientos[i] = [Desplazamientos_f[i*2],Desplazamientos_f[i*2+1],0]

Problema12 = gmsh.view.add("Problema2")
problema12 = gmsh.view.addModelData(Problema12, 0, 'PROBLEMA2', 'NodeData', NodeInfo[0], Desplazamientos  , numComponents=3)


In [11]:
gmsh.fltk.run() 